In [43]:
import matplotlib.pyplot as plt
import numpy as np
import cv2
import matplotlib; matplotlib.use('TkAgg')
import torch
import os
import kornia as K
import sys
import math
import random
import scipy
# %matplotlib inline

In [44]:
def imageRead(fileName):
    img = cv2.imread(fileName)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = (img.astype(np.float32) / 255.0)
    return img

def grayAndNormalize(img):
    
    img = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)    

    return img

def plot2Imgs(img1,img2):
    plt.figure(figsize=(20,20))
    plt.subplot(1,2,1)
    plt.title("Lt Image")
    plt.imshow(img1)
    plt.subplot(1,2,2)
    plt.title("Rt Image")
    plt.imshow(img2)

def resizeHalf(img):
    # First blur the image to remove noise
    # img = cv2.GaussianBlur(img, (3, 3), 1)
    h, w, c = img.shape
    img = cv2.resize(img, (w//2, h//2))
    return img

def resizeRatio(img, ratio):
    # First blur the image to remove noise
    # img = cv2.GaussianBlur(img, (3, 3), 1)
    h, w = img.shape
    img = cv2.resize(img, (int(w*ratio), int(h*ratio)))
    return img

def resize(img, w, h):
    # First blur the image to remove noise
    # img = cv2.GaussianBlur(img, (3, 3), 1)
    img = cv2.resize(img, (w, h))
    return img

In [45]:
def flowToColor(flow, maxflow=None, verbose=False):
    '''
    args
        flow (numpy array) height x width x 2
    return
        img_color (numpy array) height x width x 3
    '''

    UNKNOWN_FLOW_THRESH = 5e2
    eps = 1e-6

    height, width, nBands = flow.shape

    if nBands != 2:
        exit('flowToColor: image must have two bands')

    u = flow[:, :, 0]
    v = flow[:, :, 1]

    maxu = -999
    maxv = -999

    minu = 999
    minv = 999
    maxrad = -1

    # fix unknown flow
    idxUnknown = (abs(u) > UNKNOWN_FLOW_THRESH) | (abs(v) > UNKNOWN_FLOW_THRESH)
    u[idxUnknown] = 0
    v[idxUnknown] = 0

    maxu = max(maxu, u.max())
    minu = min(minu, u.min())

    maxv = max(maxv, v.max())
    minv = min(minv, v.min())

    rad = np.sqrt(u ** 2 + v ** 2)
    maxrad = max(maxrad, rad.max())

    if verbose:
        print('max flow: %.4f flow range: u = %.3f .. %.3f; v = %.3f .. %.3f\n' %
              (maxrad, minu, maxu, minv, maxv))

    if maxflow is not None:
        if maxflow > 0:
            maxrad = maxflow

    u = u / (maxrad + eps)
    v = v / (maxrad + eps)

    img = computeColor(u, v)

    # unknown flow
    # IDX = repmat(idxUnknown, [1, 1, 3])
    img[idxUnknown] = 0

    return img

def computeColor(u, v, cast_uint8=True):
    '''
    args
        u (numpy array) height x width
        v (numpy array) height x width
        cast_uint8 (bool) set False to have image range 0-1 (np.float32)
    return
        img_color (numpy array) height x width x 3
    '''

    nanIdx = np.isnan(u) | np.isnan(v)
    u[nanIdx] = 0
    v[nanIdx] = 0

    colorwheel = makeColorwheel()
    ncols = colorwheel.shape[0]

    rad = np.sqrt(u ** 2 + v ** 2)

    a = np.arctan2(-v, -u)/np.pi

    fk = (a + 1) / 2 * (ncols - 1)  # -1~1 maped to 1~ncols

    k0 = np.floor(fk).astype(int)  # 1, 2, ..., ncols

    k1 = k0 + 1
    k1[k1 == ncols + 1] = 1

    f = fk - k0

    height, width = u.shape
    img = np.zeros((height, width, 3), np.float32)
    nrows = colorwheel.shape[1]
    for i in range(nrows):
        tmp = colorwheel[:, i]
        col0 = tmp[k0.reshape(-1)] / 255
        col1 = tmp[k1.reshape(-1)] / 255
        col = col0.reshape(height, width) * (1 - f) + \
              col1.reshape(height, width) * f

        idx = rad <= 1
        col[idx] = 1 - rad[idx] * (1 - col[idx])  # increase saturation with radius

        col[np.logical_not(idx)] *= 0.75  # out of range

        img[:, :, i] = col * (1 - nanIdx)

    if cast_uint8:
        img = np.floor(img * 255).astype(np.uint8)
    return img


def makeColorwheel():
    '''
    color encoding scheme
    adapted from the color circle idea described at
    http://members.shaw.ca/quadibloc/other/colint.htm
    '''

    RY = 15
    YG = 6
    GC = 4
    CB = 11
    BM = 13
    MR = 6

    ncols = RY + YG + GC + CB + BM + MR

    colorwheel = np.zeros((ncols, 3))  # r g b

    col = 0
    # RY
    colorwheel[0:RY, 0] = 255
    colorwheel[0:RY, 1] = np.floor(255*np.array(range(RY))/RY)
    col = col+RY

    # YG
    colorwheel[col:col+YG, 0] = 255 - np.floor(255*np.array(range(YG))/YG)
    colorwheel[col:col+YG, 1] = 255
    col = col+YG

    # GC
    colorwheel[col:col+GC, 1] = 255
    colorwheel[col:col+GC, 2] = np.floor(255*np.array(range(GC))/GC)
    col = col+GC

    # CB
    colorwheel[col:col+CB, 1] = 255 - np.floor(255*np.array(range(CB))/CB)
    colorwheel[col:col+CB, 2] = 255
    col = col+CB

    # BM
    colorwheel[col:col+BM, 2] = 255
    colorwheel[col:col+BM, 0] = np.floor(255*np.array(range(BM))/BM)
    col = col+BM

    # MR
    colorwheel[col:col+MR, 2] = 255 - np.floor(255*np.array(range(MR))/MR)
    colorwheel[col:col+MR, 0] = 255

    return colorwheel

In [46]:
def myFlowPyramid(img1,img2,windowSize,threshold,pyramidLevels):
    xFiveTap = (np.array([-1, 8, 0, -8, 1]) * 1/12).reshape(1,5)    
    xFiveTap = np.flip(xFiveTap)
    yFiveTap = (np.array([-1, 8, 0, -8, 1]) * 1/12).reshape(5,1)
    yFiveTap = np.flip(yFiveTap)
    gaussian = cv2.getGaussianKernel(3,1)

    baseImg1 = img1
    baseImg2 = img2
    uFlow = np.zeros(Ix2.shape)
    vFlow = np.zeros(Ix2.shape)
    valid = np.ones(Ix2.shape)

    for i in range(pyramidLevels):
        if(i==0):
            img1 = resizeRatio(baseImg1,1/(2**(pyramidLevels-i)))
            img2 = resizeRatio(baseImg2,1/(2**(pyramidLevels-i)))
        xGrad = cv2.filter2D(img1, -1, xFiveTap)
        yGrad = cv2.filter2D(img1, -1, yFiveTap)
        tGrad = (cv2.filter2D(img2, -1, gaussian) - cv2.filter2D(img1, -1, gaussian))    
        Ix2 = xGrad * xGrad
        Iy2 = yGrad * yGrad
        Ixy = xGrad * yGrad
        Ixt = xGrad * tGrad
        Iyt = yGrad * tGrad
        windowFilter = np.ones((windowSize,windowSize))
        # windowFilter = cv2.getGaussianKernel(windowSize,0.2) * windowSize * windowSize
        Ix2 = cv2.filter2D(Ix2, -1, windowFilter)
        Iy2 = cv2.filter2D(Iy2, -1, windowFilter)
        Ixy = cv2.filter2D(Ixy, -1, windowFilter)
        Ixt = cv2.filter2D(Ixt, -1, windowFilter)
        Iyt = cv2.filter2D(Iyt, -1, windowFilter)    
        

        uFlow = np.zeros(Ix2.shape)
        vFlow = np.zeros(Ix2.shape)
        valid = np.ones(Ix2.shape)        
        for i in range(Ix2.shape[0]):
            for j in range(Ix2.shape[1]):
                ATA = np.array([[Ix2[i,j],Ixy[i,j]],[Ixy[i,j],Iy2[i,j]]])
                ATb = np.array([[-Ixt[i,j]],[-Iyt[i,j]]])

                eigenValues = np.linalg.eigvals(ATA)
                if eigenValues[0] < threshold or eigenValues[1] < threshold:
                    valid[i,j] = 0
                    # print("I didnt mult at all!")
                    continue
                try:
                    u,v = np.matmul(np.linalg.pinv(ATA),ATb)  
                    # print("I multed")              
                except:
                    # print("I didnt mult")
                    valid[i,j] = 0
                    u = 0
                    v = 0
                uFlow[i,j] = u
                vFlow[i,j] = v    
        
        # Reverse resize and scale the flow by resizeMultiple
        uFlow = resizeRatio(uFlow,1/(2**(pyramidLevels-(i+1)))) * (2**(i+1))
        vFlow = resizeRatio(vFlow,1/(2**(pyramidLevels-(i+1)))) * (2**(i+1))
        valid = resizeRatio(valid,1/(2**(pyramidLevels-(i+1))))
        img2 = resizeRatio(baseImg2,1/(2**(pyramidLevels-(i+1))))
        img1 = resizeRatio(baseImg1,1/(2**(pyramidLevels-(i+1))))
        img1 = myWarp(img1,uFlow,vFlow)        
        
        

        
        
        
    # uFlow = resize(uFlow,baseImg1.shape[1],baseImg1.shape[0]) * (1/resizeMultiple)
    # vFlow = resize(vFlow,baseImg1.shape[1],baseImg1.shape[0]) * (1/resizeMultiple)
    return uFlow,vFlow,valid    

def myFlowX(img1,img2,windowSize,threshold,resizeFactor):
    xFiveTap = (np.array([-1, 8, 0, -8, 1]) * 1/12).reshape(1,5)    
    xFiveTap = np.flip(xFiveTap)

    yFiveTap = (np.array([-1, 8, 0, -8, 1]) * 1/12).reshape(5,1)
    yFiveTap = np.flip(yFiveTap)
    gaussian = cv2.getGaussianKernel(3,1)

    baseImg1 = img1
    baseImg2 = img2
    img1 = resizeRatio(img1,resizeFactor)
    img2 = resizeRatio(img2,resizeFactor)

    xGrad = cv2.filter2D(img1, -1, xFiveTap)
    yGrad = cv2.filter2D(img1, -1, yFiveTap)
    tGrad = (cv2.filter2D(img2, -1, gaussian) - cv2.filter2D(img1, -1, gaussian))    

    Ix2 = xGrad * xGrad
    Iy2 = yGrad * yGrad
    Ixy = xGrad * yGrad
    Ixt = xGrad * tGrad
    Iyt = yGrad * tGrad
    
    plotArrayOfImages([Ix2,Iy2,Ixy,Ixt,Iyt],["Ix2","Iy2","Ixy","Ixt","Iyt"])
    plotArrayOfImages([xGrad,yGrad,tGrad],["xGrad","yGrad","tGrad"])
    w = windowSize//2

    
    uFlow = np.zeros(Ix2.shape)
    vFlow = np.zeros(Ix2.shape)
    valid = np.ones(Ix2.shape)
    for i in range(w,Ix2.shape[0]-w):
        
        for j in range(w,Ix2.shape[1]-w):
            dx2 = Ix2[i-w:i+w+1,j-w:j+w+1].flatten()
            dy2 = Iy2[i-w:i+w+1,j-w:j+w+1].flatten()
            dxy = Ixy[i-w:i+w+1,j-w:j+w+1].flatten()
            dxt = Ixt[i-w:i+w+1,j-w:j+w+1].flatten()
            dyt = Iyt[i-w:i+w+1,j-w:j+w+1].flatten()

            

            # A = np.array([[Ix2[i,j],Ixy[i,j]],[Ixy[i,j],Iy2[i,j]]])
            # b = np.array([[-Ixt[i,j]],[-Iyt[i,j]]])

            ATA = np.matmul(A.T,A)
            eigenValues = np.linalg.eigvals(ATA)
            if eigenValues[0] < threshold or eigenValues[1] < threshold:
                valid[i,j] = 0
                continue
            try:
                u,v = np.matmul(np.linalg.inv(ATA),np.matmul(A.T,b))                
            except:
                u = 0
                v = 0
                valid[i,j] = 0
            uFlow[i,j] = u
            vFlow[i,j] = v    
    
    # Reverse resize and scale the flow by resizeFactor
    uFlow = resize(uFlow,baseImg1.shape[1],baseImg1.shape[0]) * (1/resizeFactor)
    vFlow = resize(vFlow,baseImg1.shape[1],baseImg1.shape[0]) * (1/resizeFactor)
    valid = resize(valid,baseImg1.shape[1],baseImg1.shape[0]) * (1/resizeFactor)
    return uFlow,vFlow,valid

In [64]:
def plotArrayOfImages(imgs, titles, rows=1, figsize=(20,20)):
    plt.figure(figsize=figsize)
    for i in range(len(imgs)):
        plt.subplot(rows,math.ceil(len(imgs)/rows),i+1)
        plt.title(titles[i])
        plt.imshow(imgs[i],cmap='gray')
    plt.show()
    

def plot_quiver(uFlow, vFlow, valid, scale=3, step=5):
    Y, X = np.mgrid[0:uFlow.shape[0], 0:uFlow.shape[1]]
    xFlow = uFlow.copy()
    yFlow = vFlow.copy()
    # Normalize b/w -1 and 1
    if(scale !=0):
        xFlow = xFlow / np.max(np.abs(xFlow)) * scale
        yFlow = yFlow / np.max(np.abs(yFlow)) * scale

    # Only plot vectors at valid points
    mask = valid > 0

    # Sampling points for a cleaner look
    X = X[::step, ::step]
    Y = Y[::step, ::step]
    xFlow = xFlow[::step, ::step]
    yFlow = yFlow[::step, ::step]
    mask = mask[::step, ::step]

    plt.figure(figsize=(5, 5))
    plt.quiver(X[mask], Y[mask], xFlow[mask], yFlow[mask], angles='xy', scale_units='xy', scale=3)
    plt.gca().invert_yaxis()  # Invert y-axis to match image coordinate system
    plt.show()

def myFlow(img1,img2,windowSize,threshold,resizeMultiple=1):
    xFiveTap = (np.array([-1, 8, 0, -8, 1]) * 1/12).reshape(1,5)    
    xFiveTap = np.flip(xFiveTap)

    yFiveTap = (np.array([-1, 8, 0, -8, 1]) * 1/12).reshape(5,1)
    yFiveTap = np.flip(yFiveTap)
    gaussian = cv2.getGaussianKernel(3,1)

    baseImg1 = img1    
    img1 = resizeRatio(img1,resizeMultiple)
    img2 = resizeRatio(img2,resizeMultiple)

    xGrad = cv2.filter2D(img1, -1, xFiveTap)
    yGrad = cv2.filter2D(img1, -1, yFiveTap)
    tGrad = (cv2.filter2D(img2, -1, gaussian) - cv2.filter2D(img1, -1, gaussian))    

    Ix2 = xGrad * xGrad
    Iy2 = yGrad * yGrad
    Ixy = xGrad * yGrad
    Ixt = xGrad * tGrad
    Iyt = yGrad * tGrad

    windowFilter = np.ones((windowSize,windowSize))
    # sigma = 4
    # windowFilter = cv2.getGaussianKernel(windowSize,sigma) * cv2.getGaussianKernel(windowSize,sigma).T * (windowSize**2)    
    Ix2 = cv2.filter2D(Ix2, -1, windowFilter)
    Iy2 = cv2.filter2D(Iy2, -1, windowFilter)
    Ixy = cv2.filter2D(Ixy, -1, windowFilter)
    Ixt = cv2.filter2D(Ixt, -1, windowFilter)
    Iyt = cv2.filter2D(Iyt, -1, windowFilter)
    # plotArrayOfImages([Ix2,Iy2,Ixy,Ixt,Iyt],["Ix2","Iy2","Ixy","Ixt","Iyt"])
    # plotArrayOfImages([xGrad,yGrad,tGrad],["xGrad","yGrad","tGrad"])

    
    uFlow = np.zeros(Ix2.shape)
    vFlow = np.zeros(Ix2.shape)
    valid = np.ones(Ix2.shape)
    for i in range(Ix2.shape[0]):
        for j in range(Ix2.shape[1]):
            ATA = np.array([[Ix2[i,j],Ixy[i,j]],[Ixy[i,j],Iy2[i,j]]])
            ATb = np.array([[-Ixt[i,j]],[-Iyt[i,j]]])
                   
            eigenValues = np.linalg.eigvals(ATA)
            if eigenValues[0] < threshold or eigenValues[1] < threshold:
                valid[i,j] = 0                
                continue
            try:
                u,v = np.matmul(np.linalg.pinv(ATA),ATb)                  
            except:                
                valid[i,j] = 0
                u = 0
                v = 0
            uFlow[i,j] = u
            vFlow[i,j] = v    
    
    # Reverse resize and scale the flow by resizeMultiple
    uFlow = resize(uFlow,baseImg1.shape[1],baseImg1.shape[0]) * (1/resizeMultiple)
    vFlow = resize(vFlow,baseImg1.shape[1],baseImg1.shape[0]) * (1/resizeMultiple)
    valid = resize(valid,baseImg1.shape[1],baseImg1.shape[0]) 
    return uFlow,vFlow,valid   

def myWarp(img1, uFlow, vFlow):    
    height, width = img1.shape[:2]
    # Create meshgrid for the coordinates
    x, y = np.meshgrid(np.arange(width), np.arange(height))

    # Calculate the new coordinates based on the optical flow
    warped_x = np.clip(x + uFlow*-1.4, 0, width-1)
    warped_y = np.clip(y + vFlow*-1.4, 0, height-1)
    
    # Interpolate the values of img1 at the new coordinates
    # Kind 'cubic' may produce artifacts at the boundaries; consider using 'linear' if that's the case
    interpolator = scipy.interpolate.RectBivariateSpline(np.arange(width), np.arange(height), img1)
    warped_img = interpolator.ev(warped_y, warped_x)

    return warped_img

def visulizeWarp(matchImg, warpedImg, iterations=5):

    plt.ion()  # Turn on interactive mode
    fig = plt.figure(figsize=(10, 5))
    differece = matchImg - warpedImg
    plt.title("Difference")
    plt.imshow(differece, cmap='gray')
    fig.canvas.draw()
    plt.pause(2.5)
    for _ in range(iterations):
        plt.imshow(matchImg, cmap='gray', interpolation='nearest')
        plt.title("Expected Img2")
        fig.canvas.draw()
        plt.pause(0.5)         

        plt.imshow(warpedImg, cmap='gray', interpolation='nearest')
        plt.title("Warped Img1")        
        fig.canvas.draw() 
        plt.pause(0.5)  

    plt.ioff()  # Turn off interactive mode


In [48]:

def selecteinitialPoints(image,numPoints=20, threshold=0.0005):
    sigma = 1
    k = 0.04
    gaussianFilter = cv2.getGaussianKernel(2*sigma*3+1,sigma)
    dxFilter = np.array([[-1,0,1],[-1,0,1],[-1,0,1]])
    dyFilter = dxFilter.T

    Ix = cv2.filter2D(image, -1, dxFilter)
    Iy = cv2.filter2D(image, -1, dyFilter)

    Ix2 = cv2.filter2D(Ix*Ix, -1, gaussianFilter)
    Iy2 = cv2.filter2D(Iy*Iy, -1, gaussianFilter)
    Ixy = cv2.filter2D(Ix*Iy, -1, gaussianFilter)

    r = (Ix2*Iy2 - Ixy*Ixy) - k*(Ix2+Iy2)**2

    kps = nonMaxSuppression(r, 3, threshold)
    kps = kps > threshold

    # Flatten the response array and sort by corner strength
    flat_kps = r.flatten()
    indices = np.argsort(-flat_kps)  # Negative sign for descending order

    # Pick top 2*numPoints points
    top_points = []
    for idx in indices:
        y, x = np.unravel_index(idx, r.shape)
        if kps[y, x]:
            top_points.append((x, y))
            if len(top_points) == 4 * numPoints:
                break

    # Randomly select numPoints from the top points
    selected_points = random.sample(top_points, numPoints)

    return np.array(selected_points)
    

def nonMaxSuppression(inputImg, windowSize,threshold=0.0005):
    # Apply dilation to get the local maxima within the window
    dilated = cv2.dilate(inputImg, np.ones((windowSize, windowSize)))
    localMax = (inputImg == dilated)
    thresholded = (inputImg > threshold)
    outputImg = np.where(localMax & thresholded, inputImg, 0)
    return outputImg

def getWindow(img, x, y, windowSize):
    w = windowSize//2
    return img[y-w:y+w+1, x-w:x+w+1]

def getWindowFlow(window1, window2, windowSize, threshold):
    xFiveTap = (np.array([-1, 8, 0, -8, 1]) * 1/12).reshape(1,5)
    xFiveTap = np.flip(xFiveTap)
    yFiveTap = (np.array([-1, 8, 0, -8, 1]) * 1/12).reshape(5,1)
    yFiveTap = np.flip(yFiveTap)
    gaussian = cv2.getGaussianKernel(windowSize, -1)

    xGrad = cv2.filter2D(window1, -1, xFiveTap)
    yGrad = cv2.filter2D(window1, -1, yFiveTap)
    tGrad = cv2.filter2D(window2, -1, gaussian) - cv2.filter2D(window1, -1, gaussian)

    Ix2 = xGrad * xGrad
    Iy2 = yGrad * yGrad
    Ixy = xGrad * yGrad
    Ixt = xGrad * tGrad
    Iyt = yGrad * tGrad

    windowFilter = np.ones((windowSize, windowSize))
    Ix2 = cv2.filter2D(Ix2, -1, windowFilter)
    Iy2 = cv2.filter2D(Iy2, -1, windowFilter)
    Ixy = cv2.filter2D(Ixy, -1, windowFilter)
    Ixt = cv2.filter2D(Ixt, -1, windowFilter)
    Iyt = cv2.filter2D(Iyt, -1, windowFilter)

    # Assuming the window is centered around the keypoint, find the center index
    i, j = windowSize // 2, windowSize // 2
    ATA = np.array([[Ix2[i,j], Ixy[i,j]], [Ixy[i,j], Iy2[i,j]]])
    ATb = np.array([[-Ixt[i,j]], [-Iyt[i,j]]])

    eigenValues = np.linalg.eigvals(ATA)
    if eigenValues.min() < threshold:
        valid = False
        u, v = 0, 0
    else:
        try:
            u, v = np.matmul(np.linalg.pinv(ATA), ATb).flatten()
            valid = True
        except:
            valid = False
            u, v = 0, 0

    return u, v, valid

def draw_transparent_circle(img, center, radius, color, alpha):
    overlay = img.copy()
    cv2.circle(overlay, center, radius, color, -1)

    # Blend with the original image
    return cv2.addWeighted(overlay, alpha, img, 1 - alpha, 0)


def tracker(imageSet, numPoints=20, windowSize=5, threshold=0.001):
    keypoints = selecteinitialPoints(imageSet[0], numPoints, threshold)
    paths = [np.array([kp]) for kp in keypoints]
    frames_with_paths = []
    colors = [(random.randint(0, 255), random.randint(0, 255), random.randint(0, 255)) for _ in range(numPoints)]
    for t in range(len(imageSet)-1):    
        image_with_path = imageSet[t].copy()
        if len(image_with_path.shape) == 2:  # If the image is grayscale, convert it to color
            # image_with_path*=255
            if image_with_path.dtype == np.float32 or image_with_path.dtype == np.float64:
            # Assuming the float image ranges from 0 to 1
                image_with_path = (255 * image_with_path).astype(np.uint8)
            image_with_path = cv2.cvtColor(image_with_path, cv2.COLOR_GRAY2BGR)
        nextKeyPoints = []
        for idx, kp in enumerate(keypoints):
            x, y = int(kp[0]), int(kp[1])
            window_prev = getWindow(imageSet[t], x, y, windowSize)
            window_next = getWindow(imageSet[t+1], x, y, windowSize)

            u,v,valid = getWindowFlow(window_prev, window_next, windowSize, 0.001)

            kp_next = kp + np.array([u, v])

            # if the new point is within the image
            if kp_next[0] > 0 and kp_next[0] < imageSet[0].shape[1] and kp_next[1] > 0 and kp_next[1] < imageSet[0].shape[0]:
                nextKeyPoints.append(kp_next)
                paths[idx] = np.vstack([paths[idx], kp_next])
                for j in range(1, len(paths[idx])):
                    cv2.line(image_with_path, tuple(paths[idx][j-1].astype(int)), tuple(paths[idx][j].astype(int)), colors[idx], 2)
                # Draw the endpoint
                image_with_path = draw_transparent_circle(image_with_path, tuple(paths[idx][-1].astype(int)), 5, colors[idx], alpha=0.5)
            else:
                nextKeyPoints.append(kp)
        keypoints = nextKeyPoints
        frames_with_paths.append(image_with_path)
    
    plt.ion()
    fig = plt.figure(figsize=(8,8))    
    for frame in frames_with_paths:
        plt.imshow(frame)
        fig.canvas.draw()
        plt.pause(0.02)        
        # If last frame, pause for 2 seconds
        if frame is frames_with_paths[-1]:
            plt.pause(2)        
        plt.clf()
    plt.ioff()




sphereImgs = ["Sequences/sphere/sphere_0.png", "Sequences/sphere/sphere_1.png"]
corridorImgs = ["Sequences/corridor/bt_0.png", "Sequences/corridor/bt_1.png"]
synthImgs = ["Sequences/synth/synth_0.png", "Sequences/synth/synth_1.png"]
sphereImgs = [imageRead(img) for img in sphereImgs]
corridorImgs = [imageRead(img) for img in corridorImgs]
synthImgs = [imageRead(img) for img in synthImgs]
hotelImgs = []
for i in range(51):
    hotelImgs.append("Hotel Sequence/hotel.seq%d.png" % i)
hotelImgs = [imageRead(img) for img in hotelImgs]

sphereImgs = [grayAndNormalize(img) for img in sphereImgs]
corridorImgs = [grayAndNormalize(img) for img in corridorImgs]
synthImgs = [grayAndNormalize(img) for img in synthImgs]
hotelImgs = [grayAndNormalize(img) for img in hotelImgs]

# Part 1


In [49]:
ImageToUse = corridorImgs
uFlow, vFlow, valid = myFlow(ImageToUse[0],ImageToUse[1],15,0.01,0.75)
flow = flowToColor(np.stack([uFlow,vFlow],axis=2))
ImageToUse = sphereImgs
uFlow2, vFlow2, valid2 = myFlow(ImageToUse[0],ImageToUse[1],15,0.01,0.6)
flow2 = flowToColor(np.stack([uFlow2,vFlow2],axis=2))
ImageToUse = synthImgs
uFlow3, vFlow3, valid3 = myFlow(ImageToUse[0],ImageToUse[1],11,0.01,0.81)
flow3 = flowToColor(np.stack([uFlow3,vFlow3],axis=2))
plt.ion()
plotArrayOfImages([uFlow,vFlow,valid,flow,uFlow2,vFlow2,valid2,flow2,uFlow3,vFlow3,valid3,flow3],["uFlow","vFlow","valid","flow","uFlow2","vFlow2","valid2","flow2","uFlow3","vFlow3","valid3","flow3"],rows=3,figsize=(10,10))
plt.pause(3.0)
plt.ioff()
plt.close()

# Part 2

In [65]:
ImageToUse = sphereImgs
uFlow, vFlow, valid = myFlow(ImageToUse[0],ImageToUse[1],15,0.01,0.75)
flow = flowToColor(np.stack([uFlow,vFlow],axis=2))
warped = myWarp(ImageToUse[0],uFlow,vFlow)
visulizeWarp(ImageToUse[1],warped,iterations=4)
plt.close()

# Part 3

In [51]:
ImageToUse = hotelImgs
tracker(ImageToUse,20,15)
plt.close()